In [9]:
!pip install joblib pandas xgboost scikit-learn gcsfs

    100% |████████████████████████████████| 61kB 2.9MB/s ta 0:00:011
  Stored in directory: /home/jovyan/.cache/pip/wheels/9f/0f/b9/5bc5222756d121ccace51ab3084a1c733380908a4e2f939038
Successfully built gcsfs
You are using pip version 19.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [55]:
import argparse
import logging
import joblib
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import datetime
import pickle
from sklearn.preprocessing import OneHotEncoder
import gcsfs

In [8]:
logging.basicConfig(format='%(message)s')
logging.getLogger().setLevel(logging.INFO)

In [113]:
PROJECT = 'mwpmltr'
BUCKET = 'mock_lc'
TRAIN_DATA_PATH = 'gs://{}/data/train.csv'.format(BUCKET)
TEST_DATA_PATH = 'gs://{}/data/test.csv'.format(BUCKET)
MODEL_PATH = '{}/models/lc_randomforest.pkl'.format(BUCKET)

RESPONSE = 'is_bad'

CAT_ENCODER_BASE = 'onehotencoder_'

PARAMS = {'max_depth': 50,
          'max_features': 25,
          'max_leaf_nodes': 445,
          'min_samples_leaf': 1,
          'min_samples_split': 0.000125,
          'min_weight_fraction_leaf': 0.0,
          'n_estimators': 2500}
RANDOM_SEED = 42

In [46]:
def read_data(path):
    df = pd.read_csv(path)
    y_df = df[RESPONSE]
    X_df = df.drop(RESPONSE, axis=1)
    return X_df, y_df

In [70]:
def process_dates(df):
        logging.info("Processing date column. Shape={}".format(df.shape))
        date_column = 'earliest_cr_line'

        a = datetime.datetime.now() # Should be fixed date in production
        df["earliest_cr_fill"] = df[date_column].fillna(str(datetime.datetime.now().strftime("%m/%d/%Y")))
        df["earliest_cr_fill"] = df["earliest_cr_fill"].apply(lambda x: expand_year(x))
        df["days_to_earliest_cr"] = df.earliest_cr_line.apply(
            lambda x: (pd.to_datetime(x, format="%m/%d/%y") - pd.datetime(1900, 1, 1)).days)

        df.drop([date_column, "earliest_cr_fill"], axis=1, inplace=True)
        return df
    
def process_text(df):
    logging.info("Processing text columns. Shape={}".format(df.shape))
    text_columns = ['Notes', 'purpose', 'emp_title']

    # Character counts
    df_with_processed_text = df.copy()
    for t in text_columns:
        df_with_processed_text[t + "_char_count"] = df[t].apply(lambda x: len(str(x)))

    df_with_processed_text.drop(text_columns, axis=1, inplace=True)
    return df_with_processed_text

def add_other_features(df):
    logging.info("Adding log features. Shape={}".format(df.shape))
    df['log_annual_inc'] = np.log(df['annual_inc'] + 1)
    df['log_mths_since_last_record'] = np.log(df['mths_since_last_record'] + 1)
    df['log_revol_bal'] = np.log(df['revol_bal'] + 1)

    return df

def process_categorical(df, is_train=True):
    logging.info("Processing categorical columns. Shape={}".format(df.shape))
    categorical_columns = ['home_ownership', 'verification_status', 'pymnt_plan', 'purpose_cat',
                           'addr_state', 'initial_list_status', 'policy_code']

    fs = gcsfs.GCSFileSystem(project=PROJECT)
    # One hot encoding
    df_new = df.copy()
    if is_train:
        for col in categorical_columns:
            enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
            enc.fit(df.loc[:, [col]])
            new_onehot_features = enc.categories_[0]
            encoded = pd.DataFrame(enc.transform(df.loc[:, [col]]))
            encoded.columns = list([col + f for f in new_onehot_features])
            df_new = pd.concat([df_new, encoded], axis=1)   

            with fs.open(BUCKET + "/encoders/" + CAT_ENCODER_BASE + col + ".pkl", 'wb') as f:
                pickle.dump(enc, f)
    else:
        for col in categorical_columns:
            with fs.open(BUCKET + "/encoders/" + CAT_ENCODER_BASE + col + ".pkl", 'rb') as f:
                enc = pickle.load(f)
                new_onehot_features = enc.categories_[0]
                encoded = pd.DataFrame(enc.transform(df.loc[:, [col]]), columns=[col + f for f in new_onehot_features])
                df_new = pd.concat([df_new, encoded], axis=1)

    df_new.drop(categorical_columns, axis=1, inplace=True)

    return df_new

def process_na(df):
    def flag_nulls(df):
        logging.info("Flagging null values")
        df_with_nulls = df.copy()
        for col in df.columns:
            if df[col].isna().sum() > 0:
                df_with_nulls[col + "_null"] = df[col].isna()
        return df_with_nulls

    logging.info("Processing nulls. Shape={}".format(df.shape))
    df = flag_nulls(df)
    df.fillna(-9999, inplace=True)
    df.emp_length = df.emp_length.apply(lambda x: 0 if x == 'na' else x)

    return df

def drop_extra(df):
    df.drop("zip_code", axis=1, inplace=True)
    return df

def expand_year(x):
    """Converts YY -> YYYY

    Assumes there are no records in the current year, which will be returned as-is
    """
    this_year = datetime.datetime.now().strftime("%y")
    if int(x[::-1][:2][::-1]) < int(this_year):
        return x[0:len(x) - 2] + "20" + x[len(x) - 2:]
    elif int(x[::-1][:2][::-1]) > int(this_year):
        return x[0:len(x) - 2] + "19" + x[len(x) - 2:]
    else:
        return x
    
def fill_columns(df, is_train):
    fs = gcsfs.GCSFileSystem(project=PROJECT)
    if is_train:
        with fs.open(BUCKET + "/encoders/colnames.pkl", 'wb') as f:
            pickle.dump(df.columns.values, f)
        return df
    else:
        with fs.open(BUCKET + "/encoders/colnames.pkl", 'rb') as f:
            features = pickle.load(f)
            
            test_cols = list(df.columns)
            cols_not_in_test = {c: 0 for c in features if c not in test_cols}
            df = df.assign(**cols_not_in_test)
            return df
        

In [71]:
def preprocess(df, is_train=True):
    try:
        df = df.drop(['Id', 'Unnamed: 0'], axis=1)
        df = process_dates(df)
        df = process_text(df)
        df = add_other_features(df)
        df = process_categorical(df, is_train)
        df = process_na(df)
        df = drop_extra(df)
        df = fill_columns(df, is_train)
        return df
    except Exception as e:
        logging.error("Could not preprocess data due to error: {}".format(e))

In [48]:
X_train, y_train = read_data(TRAIN_DATA_PATH)
X_test, y_test = read_data(TEST_DATA_PATH)

In [99]:
def train_model(X_train, y_train, X_test, y_test, params=PARAMS):
    """Train the model using Random Forest Classifier using tuning params from Bayesian Optimization"""
    model = RandomForestClassifier(random_state=RANDOM_SEED, verbose=2, n_jobs=-1, **params)
    
    logging.info("Training model...")
    model.fit(X_train, y_train)
    
    return model

def eval_model(model, X_test, y_test):
    """Evaluate test AUC"""
    predictions = model.predict_proba(X_test)
    logging.info("AUC={:0.2f}".format(roc_auc_score(y_test, predictions[:, 1])))
    

def save_model(model, model_path):
    """Store model object on GCS"""
    fs = gcsfs.GCSFileSystem(project=PROJECT)
    with fs.open(model_path, 'wb') as f:
        pickle.dump(model, f)

In [124]:
class LendingClubModel(object):
    def __init__(self):
        self.train_path = TRAIN_DATA_PATH
        self.test_path = TEST_DATA_PATH
        self.model_path = MODEL_PATH
        self.model = None
        
    def train(self):
        X_train, y_train = read_data(self.train_path)
        X_test, y_test = read_data(self.test_path)
        
        X_train = preprocess(X_train, True).to_numpy()
        X_test = preprocess(X_test, False).to_numpy()
        
        self.model = train_model(X_train, y_train, X_test, y_test)
        eval_model(self.model, X_test, y_test)
        save_model(self.model, self.model_path)
        
    def predict(self, X):
        if not self.model:
            fs = gcsfs.GCSFileSystem(project=PROJECT)
            with fs.open(self.model_path, 'rb') as f:
                self.model = joblib.load(f)
                
#         X_processed = preprocess(X, False)
        
        prediction = self.model.predict(data=X)
        
        return [[prediction.item(0), prediction.item[0]]]

In [123]:
m = LendingClubModel()
m.train()

Processing date column. Shape=(8000, 26)
Processing text columns. Shape=(8000, 26)
Adding log features. Shape=(8000, 26)
Processing categorical columns. Shape=(8000, 29)
Processing nulls. Shape=(8000, 114)
Flagging null values
Processing date column. Shape=(2000, 26)
Processing text columns. Shape=(2000, 26)
Adding log features. Shape=(2000, 26)
Processing categorical columns. Shape=(2000, 29)
Processing nulls. Shape=(2000, 114)
Flagging null values
Training model...
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.2s


building tree 1 of 2500
building tree 2 of 2500
building tree 3 of 2500
building tree 4 of 2500
building tree 5 of 2500
building tree 6 of 2500
building tree 7 of 2500
building tree 8 of 2500
building tree 9 of 2500
building tree 10 of 2500
building tree 11 of 2500
building tree 12 of 2500building tree 13 of 2500

building tree 14 of 2500
building tree 15 of 2500
building tree 16 of 2500
building tree 17 of 2500
building tree 18 of 2500
building tree 19 of 2500
building tree 20 of 2500
building tree 21 of 2500
building tree 22 of 2500
building tree 23 of 2500
building tree 24 of 2500
building tree 25 of 2500
building tree 26 of 2500
building tree 27 of 2500
building tree 28 of 2500
building tree 29 of 2500
building tree 30 of 2500
building tree 31 of 2500
building tree 32 of 2500
building tree 33 of 2500
building tree 34 of 2500
building tree 35 of 2500building tree 36 of 2500

building tree 37 of 2500
building tree 38 of 2500
building tree 39 of 2500
building tree 40 of 2500
building 

[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    0.8s


building tree 191 of 2500
building tree 192 of 2500
building tree 193 of 2500
building tree 194 of 2500
building tree 195 of 2500
building tree 196 of 2500
building tree 197 of 2500
building tree 198 of 2500
building tree 199 of 2500
building tree 200 of 2500building tree 201 of 2500

building tree 202 of 2500
building tree 203 of 2500
building tree 204 of 2500
building tree 205 of 2500
building tree 206 of 2500
building tree 207 of 2500
building tree 208 of 2500
building tree 209 of 2500
building tree 210 of 2500building tree 211 of 2500

building tree 212 of 2500building tree 213 of 2500

building tree 214 of 2500
building tree 215 of 2500
building tree 216 of 2500
building tree 217 of 2500
building tree 218 of 2500
building tree 219 of 2500
building tree 220 of 2500
building tree 221 of 2500
building tree 222 of 2500building tree 223 of 2500

building tree 224 of 2500
building tree 225 of 2500
building tree 226 of 2500
building tree 227 of 2500
building tree 228 of 2500
building tre

[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:    1.8s


...

building tree 1778 of 2500
building tree 1779 of 2500
building tree 1780 of 2500
building tree 1781 of 2500
building tree 1782 of 2500
building tree 1783 of 2500
building tree 1784 of 2500
building tree 1785 of 2500
building tree 1786 of 2500
building tree 1787 of 2500
building tree 1788 of 2500
building tree 1789 of 2500
building tree 1790 of 2500
building tree 1791 of 2500
building tree 1792 of 2500
building tree 1793 of 2500
building tree 1794 of 2500
building tree 1795 of 2500
building tree 1796 of 2500
building tree 1797 of 2500
building tree 1798 of 2500
building tree 1799 of 2500
building tree 1800 of 2500
building tree 1801 of 2500
building tree 1802 of 2500
building tree 1803 of 2500
building tree 1804 of 2500
building tree 1805 of 2500
building tree 1806 of 2500
building tree 1807 of 2500
building tree 1808 of 2500
building tree 1809 of 2500
building tree 1810 of 2500
building tree 1811 of 2500
building tree 1812 of 2500
building tree 1813 of 2500
building tree 1814 of 2500
b

[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:   12.2s



building tree 1979 of 2500
building tree 1980 of 2500
building tree 1981 of 2500
building tree 1982 of 2500
building tree 1983 of 2500
building tree 1984 of 2500building tree 1985 of 2500

building tree 1986 of 2500
building tree 1987 of 2500
building tree 1988 of 2500
building tree 1989 of 2500
building tree 1990 of 2500
building tree 1991 of 2500
building tree 1992 of 2500
building tree 1993 of 2500
building tree 1994 of 2500
building tree 1995 of 2500
building tree 1996 of 2500
building tree 1997 of 2500
building tree 1998 of 2500
building tree 1999 of 2500
building tree 2000 of 2500
building tree 2001 of 2500building tree 2002 of 2500

building tree 2003 of 2500
building tree 2004 of 2500
building tree 2005 of 2500
building tree 2006 of 2500
building tree 2007 of 2500
building tree 2008 of 2500
building tree 2009 of 2500
building tree 2010 of 2500
building tree 2011 of 2500
building tree 2012 of 2500
building tree 2013 of 2500
building tree 2014 of 2500
building tree 2015 of 2500


building tree 2293 of 2500
building tree 2294 of 2500
building tree 2295 of 2500
building tree 2296 of 2500building tree 2297 of 2500

building tree 2298 of 2500
building tree 2299 of 2500
building tree 2300 of 2500
building tree 2301 of 2500
building tree 2302 of 2500
building tree 2303 of 2500
building tree 2304 of 2500
building tree 2305 of 2500
building tree 2306 of 2500
building tree 2307 of 2500
building tree 2308 of 2500
building tree 2309 of 2500
building tree 2310 of 2500
building tree 2311 of 2500building tree 2312 of 2500

building tree 2313 of 2500
building tree 2314 of 2500
building tree 2315 of 2500
building tree 2316 of 2500
building tree 2317 of 2500
building tree 2318 of 2500
building tree 2319 of 2500
building tree 2320 of 2500
building tree 2321 of 2500
building tree 2322 of 2500building tree 2323 of 2500

building tree 2324 of 2500
building tree 2325 of 2500building tree 2326 of 2500

building tree 2327 of 2500
building tree 2328 of 2500
building tree 2329 of 2500
b

[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:   15.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 632 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 997 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1442 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 1969 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 2500 out of 2500 | elapsed:    1.1s finished
AUC=0.69


In [103]:
import os
import fairing

DOCKER_REGISTRY = 'gcr.io/{}/lending-club-job'.format(PROJECT)
PY_VERSION = '.'.join([str(x) for x in sys.version_info[0:3]])
BASE_IMAGE = 'python:{}'.format(PY_VERSION)

In [104]:
# Can use this to deploy as image, train on GKE backend
from fairing import TrainJob
from fairing.backends import KubeflowGKEBackend

In [107]:
train_job = TrainJob(LendingClubModel,
                    BASE_IMAGE,
                    input_files=['requirements.txt'],
                    docker_registry=DOCKER_REGISTRY,
                    backend=KubeflowGKEBackend())

train_job.submit()

Using preprocessor: <class 'fairing.preprocessors.function.FunctionPreProcessor'>
Using docker registry: gcr.io/mwpmltr/lending-club-job
Using builder: <class 'fairing.builders.cluster.cluster.ClusterBuilder'>
/opt/conda/lib/python3.6/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
/opt/conda/lib/python3.6/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-rhrm6 to start...
Waiting for fairing-builder-rhrm6 to start...
Waiting for fairing-builder-rhrm6 to start...
Pod started running True


INFO[0000] Downloading base image python:3.6.7
ERROR: logging before flag.Parse: E0621 16:11:35.436050       1 metadata.go:159] while reading 'google-dockercfg-url' metadata: http status code: 404 while fetching url http://metadata.google.internal./computeMetadata/v1/instance/attributes/google-dockercfg-url
ERROR: logging before flag.Parse: E0621 16:11:35.447296       1 metadata.go:142] while reading 'google-dockercfg' metadata: http status code: 404 while fetching url http://metadata.google.internal./computeMetadata/v1/instance/attributes/google-dockercfg
2019/06/21 16:11:35 No matching credentials were found, falling back on anonymous
INFO[0000] Executing 0 build triggers
INFO[0000] Unpacking rootfs as cmd RUN if [ -e requirements.txt ];then pip install --no-cache -r requirements.txt; fi requires it.
INFO[0018] Taking snapshot of full filesystem...
INFO[0026] Skipping paths under /dev, as it is a whitelisted directory
INFO[0026] Skipping paths under /etc/secrets, as it is a whitelist

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
INFO[0053] Taking snapshot of full filesystem...
INFO[0063] Skipping paths under /dev, as it is a whitelisted directory
INFO[0063] Skipping paths under /etc/secrets, as it is a whitelisted directory
INFO[0063] Skipping paths under /kaniko, as it is a whitelisted directory
INFO[0063] Skipping paths under /proc, as it is a whitelisted directory
INFO[0063] Skipping paths under /sys, as it is a whitelisted directory
INFO[0063] Skipping paths under /var/run, as it is a whitelisted directory
INFO[0076] Using files from context: [/kaniko/buildcontext/app]
INFO[0076] COPY /app/ /app/
INFO[0076] Taking snapshot of files...
2019/06/21 16:12:52 existing blob: sha256:5d71636fb824265e30ff34bf20737c9cdc4f5af28b6bce86f08215c55b89bfab
2019/06/21 16:12:52 existing blob: sha256:2eeb5ce9b9240a928b0a799f9f2601027e2c6b7525394ae5c371f124058489d7
2019/06/21 16:12:52 

Training job fairing-job-52k58 launched.
Waiting for fairing-job-52k58-pbxff to start...
Waiting for fairing-job-52k58-pbxff to start...
Waiting for fairing-job-52k58-pbxff to start...
Pod started running True


Processing date column. Shape=(8000, 26)
Processing text columns. Shape=(8000, 26)
Adding log features. Shape=(8000, 26)
Processing categorical columns. Shape=(8000, 29)
Processing nulls. Shape=(8000, 114)
Flagging null values
Processing date column. Shape=(2000, 26)
Processing text columns. Shape=(2000, 26)
Adding log features. Shape=(2000, 26)
Processing categorical columns. Shape=(2000, 29)
Processing nulls. Shape=(2000, 114)
Flagging null values
Training model...
building tree 1 of 2500
building tree 2 of 2500
building tree 3 of 2500
building tree 4 of 2500
building tree 5 of 2500
building tree 6 of 2500
building tree 7 of 2500
building tree 8 of 2500
building tree 9 of 2500
building tree 10 of 2500
building tree 11 of 2500
building tree 12 of 2500
building tree 13 of 2500
building tree 14 of 2500
building tree 15 of 2500
building tree 16 of 2500
building tree 17 of 2500
building tree 18 of 2500
building tree 19 of 2500
building tree 20 of 2500
building tree 21 of 2500
building tre

...

building tree 1562 of 2500
building tree 1563 of 2500
building tree 1564 of 2500
building tree 1565 of 2500
building tree 1566 of 2500
building tree 1567 of 2500
building tree 1568 of 2500
building tree 1569 of 2500
building tree 1570 of 2500
building tree 1571 of 2500
building tree 1572 of 2500
building tree 1573 of 2500
building tree 1574 of 2500
building tree 1575 of 2500
building tree 1576 of 2500
building tree 1577 of 2500
building tree 1578 of 2500
building tree 1579 of 2500
building tree 1580 of 2500
building tree 1581 of 2500
building tree 1582 of 2500
building tree 1583 of 2500
building tree 1584 of 2500
building tree 1585 of 2500
building tree 1586 of 2500
building tree 1587 of 2500
building tree 1588 of 2500
building tree 1589 of 2500
building tree 1590 of 2500
building tree 1591 of 2500
building tree 1592 of 2500
building tree 1593 of 2500
building tree 1594 of 2500
building tree 1595 of 2500
building tree 1596 of 2500
building tree 1597 of 2500
building tree 1598 of 2500
b

building tree 1866 of 2500
building tree 1867 of 2500
building tree 1868 of 2500
building tree 1869 of 2500
building tree 1870 of 2500
building tree 1871 of 2500
building tree 1872 of 2500
building tree 1873 of 2500
building tree 1874 of 2500
building tree 1876 of 2500
building tree 1875 of 2500
building tree 1877 of 2500
building tree 1878 of 2500
building tree 1879 of 2500
building tree 1880 of 2500
building tree 1881 of 2500
building tree 1882 of 2500
building tree 1883 of 2500
building tree 1884 of 2500
building tree 1885 of 2500
building tree 1886 of 2500
building tree 1887 of 2500
building tree 1888 of 2500
building tree 1889 of 2500
building tree 1890 of 2500
building tree 1891 of 2500
building tree 1892 of 2500
building tree 1893 of 2500
building tree 1894 of 2500
building tree 1895 of 2500
building tree 1896 of 2500
building tree 1897 of 2500
building tree 1898 of 2500
building tree 1899 of 2500
building tree 1900 of 2500
building tree 1901 of 2500
building tree 1902 of 2500
b

building tree 2170 of 2500
building tree 2171 of 2500
building tree 2172 of 2500
building tree 2173 of 2500
building tree 2174 of 2500
building tree 2175 of 2500
building tree 2176 of 2500
building tree 2177 of 2500
building tree 2178 of 2500
building tree 2179 of 2500
building tree 2180 of 2500
building tree 2181 of 2500
building tree 2182 of 2500
building tree 2183 of 2500
building tree 2184 of 2500
building tree 2185 of 2500
building tree 2186 of 2500
building tree 2187 of 2500
building tree 2188 of 2500
building tree 2189 of 2500
building tree 2190 of 2500
building tree 2191 of 2500
building tree 2192 of 2500
building tree 2193 of 2500
building tree 2194 of 2500
building tree 2195 of 2500
building tree 2196 of 2500
building tree 2197 of 2500
building tree 2198 of 2500
building tree 2199 of 2500
building tree 2200 of 2500
building tree 2201 of 2500
building tree 2202 of 2500
building tree 2203 of 2500
building tree 2204 of 2500
building tree 2205 of 2500
building tree 2206 of 2500
b

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:   17.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 632 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 997 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Don

Cleaning up job fairing-job-52k58...


In [129]:
from fairing import PredictionEndpoint
from fairing.backends import KubeflowGKEBackend

endpoint = PredictionEndpoint(LendingClubModel,
                             BASE_IMAGE,
                             input_files=['requirements.txt'],
                             docker_registry=DOCKER_REGISTRY,
                             backend=KubeflowGKEBackend())

endpoint.create()

Using preprocessor: <class 'fairing.preprocessors.function.FunctionPreProcessor'>
Using docker registry: gcr.io/mwpmltr/lending-club-job
Using builder: <class 'fairing.builders.cluster.cluster.ClusterBuilder'>
/opt/conda/lib/python3.6/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
/opt/conda/lib/python3.6/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-kjpc2 to start...
Waiting for fairing-builder-kjpc2 to start...
Pod started running True


INFO[0000] Downloading base image python:3.6.7
ERROR: logging before flag.Parse: E0621 16:34:15.460544       1 metadata.go:142] while reading 'google-dockercfg' metadata: http status code: 404 while fetching url http://metadata.google.internal./computeMetadata/v1/instance/attributes/google-dockercfg
ERROR: logging before flag.Parse: E0621 16:34:15.464537       1 metadata.go:159] while reading 'google-dockercfg-url' metadata: http status code: 404 while fetching url http://metadata.google.internal./computeMetadata/v1/instance/attributes/google-dockercfg-url
2019/06/21 16:34:15 No matching credentials were found, falling back on anonymous
INFO[0000] Executing 0 build triggers
INFO[0000] Unpacking rootfs as cmd RUN if [ -e requirements.txt ];then pip install --no-cache -r requirements.txt; fi requires it.
INFO[0019] Taking snapshot of full filesystem...
INFO[0027] Skipping paths under /dev, as it is a whitelisted directory
INFO[0027] Skipping paths under /etc/secrets, as it is a whitelist

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
INFO[0054] Taking snapshot of full filesystem...
INFO[0063] Skipping paths under /dev, as it is a whitelisted directory
INFO[0063] Skipping paths under /etc/secrets, as it is a whitelisted directory
INFO[0063] Skipping paths under /kaniko, as it is a whitelisted directory
INFO[0063] Skipping paths under /proc, as it is a whitelisted directory
INFO[0063] Skipping paths under /sys, as it is a whitelisted directory
INFO[0064] Skipping paths under /var/run, as it is a whitelisted directory
INFO[0077] Using files from context: [/kaniko/buildcontext/app]
INFO[0077] COPY /app/ /app/
INFO[0077] Taking snapshot of files...
2019/06/21 16:35:33 existing blob: sha256:54f7e8ac135a5f502a6ee9537ef3d64b1cd2fa570dc0a40b4d3b6f7ac81e7486
2019/06/21 16:35:33 existing blob: sha256:d6341e30912f12f56e18564a3b582853f65376766f5f9d641a68a724ed6db88f
2019/06/21 16:35:33 

Endpoint fairing-deployer-22rdw launched.


Waiting for prediction endpoint to come up...


Prediction endpoint: http://34.66.116.252:5000/predict


In [121]:
test = pd.read_csv(TEST_DATA_PATH)
test_X = test.drop(RESPONSE, axis=1)

In [127]:
X_test = preprocess(test_X, False)
X_test = X_test.to_numpy()

Processing date column. Shape=(2000, 26)
Processing text columns. Shape=(2000, 26)
Adding log features. Shape=(2000, 26)
Processing categorical columns. Shape=(2000, 29)
Processing nulls. Shape=(2000, 114)
Flagging null values


In [131]:
predictions = endpoint.predict_nparray(X_test)
predictions.head()

ConnectionError: HTTPConnectionPool(host='34.66.116.252', port=5000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9a1a96d6d8>: Failed to establish a new connection: [Errno 111] Connection refused',))